# Manipulação e Análise dos dados

In [1]:
import pandas as pd
import sqlite3

In [2]:
conn = sqlite3.connect('/home/cristina/Public/projetos/flight_fidelity/db/database.db')

## 1. União de tabelas

In [3]:
check_flight = """
    SELECT *
    FROM flight_activity
"""

In [4]:
db1 = pd.read_sql_query(check_flight, conn)
db1.head()

,loyalty_number,year,month,flights_booked,flights_with_companions,total_flights,distance,points_accumulated,points_redeemed,dollar_cost_points_redeemed
0,100018,2017,1,3,0,3,1521,152.0,0,0
1,100102,2017,1,10,4,14,2030,203.0,0,0
2,100140,2017,1,6,0,6,1200,120.0,0,0
3,100214,2017,1,0,0,0,0,0.0,0,0
4,100272,2017,1,0,0,0,0,0.0,0,0


In [5]:
check_loyalty = """
    SELECT *
    FROM flight_loyalty_history
"""

In [6]:
db2 = pd.read_sql_query(check_loyalty, conn)
db2.head()

,loyalty_number,country,province,city,postal_code,gender,education,salary,marital_status,loyalty_card,clv,enrollment_type,enrollment_year,enrollment_month,cancellation_year,cancellation_month
0,480934,Canada,Ontario,Toronto,M2Z 4K1,Female,Bachelor,83236.0,Married,Star,3839.14,Standard,2016,2,NaN,NaN
1,549612,Canada,Alberta,Edmonton,T3G 6Y6,Male,College,NaN,Divorced,Star,3839.61,Standard,2016,3,NaN,NaN
2,429460,Canada,British Columbia,Vancouver,V6E 3D9,Male,College,NaN,Single,Star,3839.75,Standard,2014,7,2018.0,1.0
3,608370,Canada,Ontario,Toronto,P1W 1K4,Male,College,NaN,Single,Star,3839.75,Standard,2013,2,NaN,NaN
4,530508,Canada,Quebec,Hull,J8Y 3Z5,Male,Bachelor,103495.0,Married,Star,3842.79,Standard,2014,10,NaN,NaN


In [7]:
join_table = """
    SELECT *
    FROM 
        flight_activity fa LEFT JOIN flight_loyalty_history flh ON (fa.loyalty_number = flh.loyalty_number)
"""

In [8]:
dt = pd.read_sql_query(join_table, conn)
dt.head()

,loyalty_number,year,month,flights_booked,flights_with_companions,total_flights,distance,points_accumulated,points_redeemed,dollar_cost_points_redeemed,...,education,salary,marital_status,loyalty_card,clv,enrollment_type,enrollment_year,enrollment_month,cancellation_year,cancellation_month
0,100018,2017,1,3,0,3,1521,152.0,0,0,...,Bachelor,92552.0,Married,Aurora,7919.20,Standard,2016,8,NaN,NaN
1,100102,2017,1,10,4,14,2030,203.0,0,0,...,College,NaN,Single,Nova,2887.74,Standard,2013,3,NaN,NaN
2,100140,2017,1,6,0,6,1200,120.0,0,0,...,College,NaN,Divorced,Nova,2838.07,Standard,2016,7,NaN,NaN
3,100214,2017,1,0,0,0,0,0.0,0,0,...,Bachelor,63253.0,Married,Star,4170.57,Standard,2015,8,NaN,NaN
4,100272,2017,1,0,0,0,0,0.0,0,0,...,Bachelor,91163.0,Divorced,Star,6622.05,Standard,2014,1,NaN,NaN


## 2. Análise dos dados

In [9]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 405624 entries, 0 to 405623
Data columns (total 26 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   loyalty_number               405624 non-null  int64  
 1   year                         405624 non-null  int64  
 2   month                        405624 non-null  int64  
 3   flights_booked               405624 non-null  int64  
 4   flights_with_companions      405624 non-null  int64  
 5   total_flights                405624 non-null  int64  
 6   distance                     405624 non-null  int64  
 7   points_accumulated           405624 non-null  float64
 8   points_redeemed              405624 non-null  int64  
 9   dollar_cost_points_redeemed  405624 non-null  int64  
 10  loyalty_number               405624 non-null  int64  
 11  country                      405624 non-null  object 
 12  province                     405624 non-null  object 
 13 

In [14]:
def join_query(table):
    df = pd.read_sql_query(table, conn)
    return df.head()

In [19]:
select_star = """
    SELECT 
        fa.loyalty_number,
        flh.gender,
        flh.city,
        flh.loyalty_card
    FROM 
        flight_activity fa LEFT JOIN flight_loyalty_history flh ON (fa.loyalty_number = flh.loyalty_number)
    WHERE
        flh.loyalty_card = 'Star' AND fa.distance = 0
"""

In [20]:
join_query(select_star)

,loyalty_number,gender,city,loyalty_card
0,480934,Female,Toronto,Star
1,480934,Female,Toronto,Star
2,480934,Female,Toronto,Star
3,480934,Female,Toronto,Star
4,549612,Male,Edmonton,Star


In [21]:
select_aurora = """
    SELECT 
        fa.loyalty_number,
        flh.gender,
        flh.city,
        fa.total_flights,
        flh.loyalty_card
    FROM 
        flight_activity fa LEFT JOIN flight_loyalty_history flh ON (fa.loyalty_number = flh.loyalty_number)
    WHERE
        flh.loyalty_card = 'Aurora' 
        AND flh.gender = 'Female' 
        AND flh.city = 'Toronto'
        AND fa.total_flights > 30
"""

In [22]:
join_query(select_aurora)

,loyalty_number,gender,city,total_flights,loyalty_card
0,929701,Female,Toronto,31,Aurora
1,975749,Female,Toronto,31,Aurora


In [23]:
dt['education'].unique()

array(['Bachelor', 'College', 'Master', 'High School or Below', 'Doctor'],
      dtype=object)

In [24]:
select_salary = """
    SELECT 
        fa.loyalty_number,
        flh.gender,
        fa.points_accumulated,
        flh.loyalty_card
    FROM 
        flight_activity fa LEFT JOIN flight_loyalty_history flh ON (fa.loyalty_number = flh.loyalty_number)
    WHERE
        flh.salary > 13200
        AND flh.marital_status = 'Married'
        AND flh.education = 'Master'
        AND fa.flights_booked = fa.total_flights
"""

In [25]:
join_query(select_salary)

,loyalty_number,gender,points_accumulated,loyalty_card
0,205785,Male,0.0,Star
1,205785,Male,0.0,Star
2,205785,Male,0.0,Star
3,205785,Male,0.0,Star
4,205785,Male,0.0,Star


In [26]:
dt['distance'].sum()

490350765

In [27]:
dt['salary'].mean()

79268.82595262615

In [28]:
dt['points_accumulated'].sum()

50172736.25

In [29]:
conn.close()